In [1]:
import os, sys
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="1,2" 
import keras
import sent2vec
import seq2seq
from seq2seq.models import AttentionSeq2Seq
from seq2seq.models import Seq2Seq
from keras.utils import multi_gpu_model
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import shutil
import pickle
import tqdm
from tqdm import tqdm
import math

import numpy as np
from random import shuffle
from math import log, floor
import pandas as pd
import tensorflow as tf
import tensorboard as tb
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
import gensim
from gensim.models.word2vec import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *
from keras.utils import *
from sklearn.model_selection import *
import random
from random import shuffle
import re
from operator import itemgetter
from keras.utils.generic_utils import *
from keras import regularizers
import string
import unicodedata as udata
import pickle
from keras.applications import *
from keras.preprocessing.image import *
import pause, datetime
import matplotlib.pyplot as plt
from sklearn.naive_bayes import *
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk import word_tokenize
from nltk.stem.porter import *
from nltk.tokenize import RegexpTokenizer
from collections import Counter
import functools

Using TensorFlow backend.


In [2]:
training = pd.read_csv('data/reg_data_token.csv')
voc_list = training.text.tolist()
voc_list_str = [str(x) for x in voc_list]
# voc_list_len = [len(x) for x in voc_list_str]
voc_string = ''.join(voc_list_str)
voc_set = set(voc_string.split(' '))
try:
    voc_set.remove('')
except:
    pass
print('   ',len(voc_set))
voc_set

    111023


{'cypherpunk',
 'scowl',
 'ilovaisk',
 'khafi',
 'nayarit',
 'dalkey',
 'ncaa',
 'onu',
 'burj',
 'shantal',
 'leftwich',
 'niezal',
 'waken',
 'clearer',
 'mauriupol',
 'vormachtstellung',
 'irvington',
 'segurament',
 'mazi',
 'lewington',
 'backblast',
 'stesso',
 'escuela',
 'kasperski',
 'steingart',
 'shrew',
 'iucfcrsg',
 'agrichem',
 'julianna',
 'longev',
 'roywoodjr',
 'howarth',
 'preshow',
 'ruder',
 'yoon',
 'raatz',
 'balibous',
 'cisgend',
 'jamon',
 'day',
 'hirsch',
 'latham',
 'lang',
 'wootan',
 'praxi',
 'bylock',
 'sternshowaddict',
 'mari',
 'apm',
 'sohr',
 'feticid',
 'alimov',
 'beigebracht',
 'cynergi',
 'elzeyadi',
 'baccellieri',
 'paladio',
 'humblebrag',
 'brade',
 'achil',
 'flourish',
 'opep',
 'ecuatoriano',
 'interestingli',
 'sanctifi',
 'redict',
 'genotox',
 'monetari',
 'asidero',
 'twattish',
 'horst',
 'spackman',
 'uresti',
 'poliv',
 'margaeri',
 'conquistar',
 'ballcap',
 'oteachjohn',
 'jostl',
 'noncompetit',
 'qrator',
 'lamarr',
 'grazziot

In [3]:
# print(np.mean(voc_list_len) , np.std(voc_list_len) , np.min(voc_list_len) , np.max(voc_list_len) )
# np.quantile(voc_list_len,0.25) , np.quantile(voc_list_len,0.5) , np.quantile(voc_list_len,0.75)

1539.312791613646 2361.826171146817 2 62702


(65.0, 695.0, 2373.0)

In [24]:
# np.mean(voc_list_len) , len(voc_list_len)

(1539.312791613646, 33006)

## Transfer words to Number

In [ ]:
# voc_list = list(voc_set)
# num_df = pd.DataFrame(data={'term':voc_list})
# train_X = []
# for news in tqdm(voc_list_str):
#     trainX = []
#     for term in news:
#         try:
#             idx = num_df[num_df['term'] == term].index[0]
#         except:
#             pass
#         trainX.append(idx)
#     train_X.append(np.array(trainX))
# #     num_df
# # num_df[num_df['term'] == 'ck'].index[0]

  0%|          | 9/33006 [02:39<162:33:07, 17.73s/it]

In [3]:
tokenizer = Tokenizer(num_words=None)
tokenizer.fit_on_texts(voc_list_str)
tokens_enc = tokenizer.texts_to_sequences(voc_list_str)
tokens_enc_len = [len(tokens) for tokens in tokens_enc]
print(np.mean(tokens_enc_len) , np.std(tokens_enc_len) , np.min(tokens_enc_len) , np.max(tokens_enc_len) )
np.quantile(tokens_enc_len,0.25) , np.quantile(tokens_enc_len,0.5) , np.quantile(tokens_enc_len,0.75) , np.mean(tokens_enc_len)
# mean = 237
# iqr = 885/887.5

235.94686105325795 366.8819509536414 1 10268


(10.0, 105.0, 361.0, 235.94686105325795)

In [4]:
for i in range(100):
    print('   ',np.quantile(tokens_enc_len,i*0.01))

    1.0
    2.0
    4.0
    4.0
    5.0
    5.0
    5.0
    6.0
    6.0
    6.0
    6.0
    7.0
    7.0
    7.0
    7.0
    8.0
    8.0
    8.0
    8.0
    9.0
    9.0
    9.0
    9.0
    9.0
    10.0
    10.0
    10.0
    11.0
    11.0
    11.0
    12.0
    12.0
    12.0
    13.0
    13.0
    14.0
    14.0
    15.0
    16.0
    17.0
    18.0
    20.0
    26.779999999998836
    40.0
    53.0
    62.05000000000109
    71.0
    80.0
    89.0
    97.0
    105.0
    114.0
    122.0
    130.0
    139.0
    147.0
    155.0
    164.0
    171.0
    179.0
    187.0
    196.0
    204.0
    214.0
    224.0
    234.85000000000218
    244.0
    255.0
    266.0
    279.0
    292.0
    306.0
    318.0
    332.0
    346.0
    361.0
    375.0
    390.0
    407.0
    422.0
    437.0
    454.0
    471.0
    490.0
    509.0
    527.0
    546.0
    565.0
    584.0
    604.0
    623.0
    646.0
    670.0
    697.3700000000026
    732.0
    775.5500000000029
    836.0
    940.0
    1106.8199999999997
    153

In [5]:
np.quantile(tokens_enc_len,0.9655)

885.0

In [6]:
trainY = training.label.tolist()
len(trainY) , len(tokens_enc)

(33610, 33610)

In [7]:
max_len = 237
train_X = []
train_Y = []
for tokens,label in zip(tokens_enc,trainY):
    length = len(tokens)
    iter_ = int(length / max_len)
    if iter_ > 0:
        temp = tokens
        for it in range(iter_):
#             bound = (it+1)*max_len
            pre = temp[:max_len]
            temp = temp[max_len:]
#             print('   ',len(pre))
            train_X.append(pre)
            train_Y.append(label)
            if (len(temp) <= max_len) and (len(temp)>=9):
#                 print('   ',len(temp))
                train_X.append(temp)
                train_Y.append(label)
    else:
        if len(tokens) >= 9:
            train_X.append(tokens)
            train_Y.append(label)
len(train_X) , len(train_Y)        

(49933, 49933)

In [8]:
len_ = [len(x) for x in train_X]
np.max(len_) , np.min(len_)

(237, 9)

In [9]:
train_X = pad_sequences(train_X, maxlen=max_len, padding= 'post' )
train_Y = np.array(train_Y)
train_X.shape , train_Y.shape

((49933, 237), (49933,))

In [144]:
# train_X = pad_sequences(tokens_enc, maxlen=361, padding= 'post' )
# train_Y = np.array(trainY)
# train_X.shape , train_Y.shape

((33610, 361), (33610,))

In [10]:
np.max(train_X)

110894

In [11]:
def split_valid_set(X_all, Y_all, percentage):
    all_data_size = len(X_all)
    valid_data_size = int(floor(all_data_size * percentage))

    X_all, Y_all = _shuffle(X_all, Y_all)

    X_train, Y_train = X_all[0:valid_data_size], Y_all[0:valid_data_size]
    X_valid, Y_valid = X_all[valid_data_size:], Y_all[valid_data_size:]

    return X_train, Y_train, X_valid, Y_valid

In [12]:
def _shuffle(X, Y):
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
#     print(X.shape, Y.shape)
    return (X[randomize], Y[randomize])

In [2]:
train_X, train_Y, valid_X, valid_Y = split_valid_set(train_X, train_Y, 0.9)

NameError: name 'split_valid_set' is not defined

Resume

In [3]:
# pickle.dump(obj=(train_X,train_Y,valid_X,valid_Y),file=open('data/train_set.pkl','wb'))
train_X, train_Y, valid_X, valid_Y = pickle.load(open('data/train_set.pkl','rb'))

## Regression model

In [10]:
max_len = train_X.shape[1]

inputs = Input(shape=(max_len,))
emb_vec = Embedding(input_dim=np.max(train_X),output_dim=128,input_length=max_len)(inputs)
conv = Conv1D(64,kernel_size=(9,), strides=1, padding='causal', data_format='channels_last')(emb_vec)
bn = BatchNormalization()(conv)
BiLSTM = Bidirectional(LSTM(64,return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(bn)#64
bn = BatchNormalization()(BiLSTM)
rnn = Bidirectional(GRU(64,return_sequences=False,dropout=0.2,recurrent_dropout=0.2))(bn) #64
bn1 = BatchNormalization()(rnn) # +FC*3? selu? 64? 3rd Dense shortcut from rnn/? kernel_initializer? dropout
dense = Dense(128,activation='selu',kernel_initializer='lecun_normal')(bn1)
bn2 = BatchNormalization()(dense)
do = Dropout(0.3)(bn2)
# bn = Concatenate()([do,bn1])
# dense = Dense(64,activation='selu',kernel_initializer='lecun_normal')(bn)
# do = Dropout(0.4)(dense)
# bn = BatchNormalization()(do)
# bn = Concatenate()([bn,bn2])
# dense = Dense(64,activation='selu',kernel_initializer='lecun_normal')(bn)
# do = Dropout(0.5)(dense)
# bn = BatchNormalization()(do)
output = Dense(1,activation='sigmoid')(do)

model = Model(inputs,output)
# model = multi_gpu_model(model,gpus=2)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 237)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 237, 128)          14194432  
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 237, 64)           73792     
_________________________________________________________________
batch_normalization_10 (Batc (None, 237, 64)           256       
_________________________________________________________________
bidirectional_5 (Bidirection (None, 237, 128)          66048     
_________________________________________________________________
batch_normalization_11 (Batc (None, 237, 128)          512       
_________________________________________________________________
bidirectional_6 (Bidirection (None, 128)               74112     
__________

## FIT

1st time

In [12]:
opt=Adam() #Nadam()
batchSize=2048 #32
patien=25
epoch=1000
# hidden_dims=128
# io_dim=128
# input_lengths=train_X.shape[1] #profile_Q3
# output_lengths= train_Y2.shape[1]#rep_max size
# depths=1
# dp = 0.01
saveP = 'model/Reg_keras2.h5' #1: 00010: val_mean_absolute_error improved from 0.10455 to 0.09934, saving model to model/Reg_keras.h5
logD = './model/logs/'
history = History()
print("input:",train_X.shape[1])#,'output_length:',train_Y.shape[1])

input: 237


In [13]:
model.compile(optimizer=opt, loss='mse', metrics=['mae'])
callback=[
    ReduceLROnPlateau(monitor='loss', factor=0.5, patience=int(patien/3),min_lr=1e-6,mode='min' ),
    EarlyStopping(patience=patien,monitor='val_loss',verbose=1),
    ModelCheckpoint(saveP,monitor='val_mean_absolute_error',verbose=1,save_best_only=True, save_weights_only=True),
    TensorBoard(log_dir=logD), 
    history,
]
model.fit(train_X, train_Y,
                epochs=epoch,
                batch_size=batchSize,
                shuffle=True,
                validation_data=(valid_X, valid_Y),
                callbacks=callback, 
#                 class_weight='auto'
                )
model.save(saveP+"_all.h5") #184sec/0.04/0.19
# 127s 3ms/step - loss: 0.0072 - mean_absolute_error: 0.0380 - val_loss: 0.0920 - val_mean_absolute_error: 0.1414

Train on 44939 samples, validate on 4994 samples
Epoch 1/1000
44939/44939 [==============================] - 96s 2ms/step - loss: 0.1989 - mean_absolute_error: 0.3391 - val_loss: 0.1343 - val_mean_absolute_error: 0.1827

Epoch 00001: val_mean_absolute_error improved from inf to 0.18269, saving model to model/Reg_keras2.h5
Epoch 2/1000
44939/44939 [==============================] - 81s 2ms/step - loss: 0.0768 - mean_absolute_error: 0.1461 - val_loss: 0.0853 - val_mean_absolute_error: 0.1331

Epoch 00002: val_mean_absolute_error improved from 0.18269 to 0.13308, saving model to model/Reg_keras2.h5
Epoch 3/1000
44939/44939 [==============================] - 80s 2ms/step - loss: 0.0493 - mean_absolute_error: 0.1031 - val_loss: 0.0801 - val_mean_absolute_error: 0.1221

Epoch 00003: val_mean_absolute_error improved from 0.13308 to 0.12208, saving model to model/Reg_keras2.h5
Epoch 4/1000
44939/44939 [==============================] - 79s 2ms/step - loss: 0.0363 - mean_absolute_error: 0.0805 

2nd time
* fix back lyers
* lower lr
* lower patience

In [14]:
opt=Nadam(lr=0.0005) 
batchSize=512
patien=15
epoch=100
# hidden_dims=128
# io_dim=128
# input_lengths=train_X.shape[1] #profile_Q3
# output_lengths= train_Y2.shape[1]#rep_max size
# depths=1
# dp = 0.01
saveP = 'model/Reg_keras2.h5' #2: val_mean_absolute_error did not improve from 0.10362
logD = './model/logs/'
history = History()
print("input:",train_X.shape[1])#,'output_length:',train_Y.shape[1])

input: 237


In [15]:
# model = load_model(saveP+"_all.h5")
model.load_weights(saveP)
model = multi_gpu_model(model,gpus=2)
model.compile(optimizer=opt, loss='mse', metrics=['mae'])

callback=[
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=int(patien/1.5),min_lr=1e-6,mode='min' ),
    EarlyStopping(patience=patien,monitor='val_loss',verbose=1),
    ModelCheckpoint(saveP,monitor='val_mean_absolute_error',verbose=1,save_best_only=True, save_weights_only=True),
    TensorBoard(log_dir=logD), 
    history,
]
model.fit(train_X, train_Y,
                epochs=epoch,
                batch_size=batchSize,
                shuffle=True,
                validation_data=(valid_X, valid_Y),
                callbacks=callback, 
                class_weight='auto'
                )
model.save(saveP+"_all.h5")

Train on 44939 samples, validate on 4994 samples
Epoch 1/100
44939/44939 [==============================] - 303s 7ms/step - loss: 0.0175 - mean_absolute_error: 0.0462 - val_loss: 0.0694 - val_mean_absolute_error: 0.1047

Epoch 00001: val_mean_absolute_error improved from inf to 0.10473, saving model to model/Reg_keras2.h5
Epoch 2/100
44939/44939 [==============================] - 291s 6ms/step - loss: 0.0153 - mean_absolute_error: 0.0428 - val_loss: 0.0735 - val_mean_absolute_error: 0.1073

Epoch 00002: val_mean_absolute_error did not improve from 0.10473
Epoch 3/100
44939/44939 [==============================] - 373s 8ms/step - loss: 0.0137 - mean_absolute_error: 0.0400 - val_loss: 0.0704 - val_mean_absolute_error: 0.1056

Epoch 00003: val_mean_absolute_error did not improve from 0.10473
Epoch 4/100
44939/44939 [==============================] - 296s 7ms/step - loss: 0.0111 - mean_absolute_error: 0.0352 - val_loss: 0.0704 - val_mean_absolute_error: 0.1063

Epoch 00004: val_mean_absolu

44939/44939 [==============================] - 292s 7ms/step - loss: 0.0017 - mean_absolute_error: 0.0146 - val_loss: 0.0672 - val_mean_absolute_error: 0.1030

Epoch 00034: val_mean_absolute_error did not improve from 0.10238
Epoch 35/100
44939/44939 [==============================] - 295s 7ms/step - loss: 0.0014 - mean_absolute_error: 0.0137 - val_loss: 0.0672 - val_mean_absolute_error: 0.1020

Epoch 00035: val_mean_absolute_error improved from 0.10238 to 0.10202, saving model to model/Reg_keras2.h5
Epoch 36/100
44939/44939 [==============================] - 293s 7ms/step - loss: 0.0014 - mean_absolute_error: 0.0137 - val_loss: 0.0681 - val_mean_absolute_error: 0.1033

Epoch 00036: val_mean_absolute_error did not improve from 0.10202
Epoch 37/100
44939/44939 [==============================] - 296s 7ms/step - loss: 0.0014 - mean_absolute_error: 0.0135 - val_loss: 0.0683 - val_mean_absolute_error: 0.1041

Epoch 00037: val_mean_absolute_error did not improve from 0.10202
Epoch 38/100
449

In [18]:
model2 = load_model('model/Reg_keras2.h5_all.h5')
model2.load_weights('model/Reg_keras2.h5')
model2.evaluate(valid_X,valid_Y)

4994/4994 [==============================] - 210s 42ms/step


[0.06722560542694177, 0.10202433797875833]

In [146]:
model.compile(loss='mse', optimizer='rmsprop')
model.fit(train_X,train_Y,epochs=50,batch_size=512) #GRU-BI，361

Epoch 1/50
33610/33610 [==============================] - 81s 2ms/step - loss: 0.1581
Epoch 2/50
33610/33610 [==============================] - 79s 2ms/step - loss: 0.1028
Epoch 3/50
33610/33610 [==============================] - 79s 2ms/step - loss: 0.0908
Epoch 4/50
33610/33610 [==============================] - 79s 2ms/step - loss: 0.0814
Epoch 5/50
33610/33610 [==============================] - 79s 2ms/step - loss: 0.0746
Epoch 6/50
33610/33610 [==============================] - 79s 2ms/step - loss: 0.0671
Epoch 7/50
33610/33610 [==============================] - 79s 2ms/step - loss: 0.0606
Epoch 8/50
33610/33610 [==============================] - 80s 2ms/step - loss: 0.0550
Epoch 9/50
33610/33610 [==============================] - 79s 2ms/step - loss: 0.0507
Epoch 10/50
33610/33610 [==============================] - 79s 2ms/step - loss: 0.0466
Epoch 11/50
33610/33610 [==============================] - 79s 2ms/step - loss: 0.0433
Epoch 12/50
33610/33610 [===========================

In [130]:
model.compile(loss='mse', optimizer='rmsprop')
model.fit(train_X,train_Y) #GRU-BI，237

Epoch 1/1
49933/49933 [==============================] - 1237s 25ms/step - loss: 0.0943


In [143]:
model.compile(loss='mse', optimizer='rmsprop')
model.fit(train_X,train_Y) #LSTM-BI、885

Epoch 1/1
33610/33610 [==============================] - 3810s 113ms/step - loss: 0.1102


In [128]:
model.compile(loss='mse', optimizer='rmsprop')
model.fit(train_X,train_Y) #LSTM-BI、237

Epoch 1/1
49933/49933 [==============================] - 1541s 31ms/step - loss: 0.0878


In [150]:
model.compile(loss='mse', optimizer='rmsprop')
model.fit(train_X,train_Y,epochs=50,batch_size=512) #LSTM-BI、237

Epoch 1/50
49933/49933 [==============================] - 80s 2ms/step - loss: 0.1553
Epoch 2/50
49933/49933 [==============================] - 78s 2ms/step - loss: 0.0988
Epoch 3/50
49933/49933 [==============================] - 79s 2ms/step - loss: 0.0762
Epoch 4/50
49933/49933 [==============================] - 79s 2ms/step - loss: 0.0539
Epoch 5/50
49933/49933 [==============================] - 78s 2ms/step - loss: 0.0459
Epoch 6/50
49933/49933 [==============================] - 78s 2ms/step - loss: 0.0351
Epoch 7/50
49933/49933 [==============================] - 79s 2ms/step - loss: 0.0291
Epoch 8/50
49933/49933 [==============================] - 78s 2ms/step - loss: 0.0257
Epoch 9/50
49933/49933 [==============================] - 79s 2ms/step - loss: 0.0232
Epoch 10/50
49933/49933 [==============================] - 79s 2ms/step - loss: 0.0208
Epoch 11/50
49933/49933 [==============================] - 79s 2ms/step - loss: 0.0195
Epoch 12/50
49933/49933 [===========================

In [17]:
model = Sequential()
model.add(Conv1D(input_shape=(None, N_features)))